# 연관규칙 분석(Association Rule Analysis)

- 아이템과 아이템 간의 상호 관게나 종속 관계를 찾아내는 분석 방법
    - 유통업에서 마케팅을 위해 활발하게 사용
        - 유통업에서는 장바구니 분석(Basket Analysis)로 부르기도 함
        - POS 시스템 도입 후 더욱 폭넓게 활용
        - 상품 구매 예측, 상품 추천, 사은품이나 패키지 구성 등
    - 다양한 분야에 적용 가능
        - 호텔/숙박업, 금융사, 보험사 등등

# 연관규칙을 찾기 위한 지표

- 신뢰도(Confidence)
    - X -> Y 의 신뢰도란 X를 포함하는 거래 내역 중 Y도 포함된 비율
        - 씨리얼 -> 우유의 신뢰도가 높으려면 씨리얼을 구매한 고객이 우유를 함께 구매한 비율이 높아야함
    - $P(Y|X)로 표시$
        - $P(Y|X) = P(X \cap Y) / P(X)$ 로 계산
    - 신뢰도에는 방향성이 존재
        - 사과 -> 우유의 신뢰도 : 66.7%
        - 우유 -> 사과의 신뢰도 : 50%
        - 우유를 구매한 사람에게 씨리얼을 권하는 것과 씨리얼을 구매한 사람에게 우유를 권하는 것은 다름
      

- 지지도(Support)
    - 전체 구매 건 중 차지하는 비율
        - 빈도가 높은 조합을 찾기 위함(빈도가 너무 낮으면 의미X)
    - $P(X\cap Y) / 전체$ 로 계산 전체는 1이므로 생략 가능
    - 지지도가 일정 수준을 넘지 못하면 두 물품간의 연관관계가 아주 높더라도 실질적인 의미가 없다는 의미로 사용됨

- 향상도(Lift)
    - Y상품이 얼마나 많이 팔리는지에 대한 확률 대비 X를 구매한 사람 중 Y도 함께 구매한 사람의 비율(신뢰도)
        - X를 구매하지 않고 Y를 구매한 사람의 비율이 높을수록 X와 Y의 조합은 별 의미가 없어짐
        - 생수는 마트에서 가장 많이 팔려 다른 물품과 묶는 의미가 없다.
        
    - 지지도의 취지는 빈도가 낮은 상품을 버리기 위한 지표
    - 향상도는 의미있는 조합을 골라낼 때 활용하기 위한 지표

현업에서의 연관규칙 분석 시 모든 품목에 대한 신뢰도, 지지도, 향상도 계산을 불가능하다.

따라서 지지도를 기준으로 먼저 필터링 한 후 신뢰도와 지지도로 선별한다.

# Apriori Algorithm

In [1]:
! pip install mixtend

ERROR: Could not find a version that satisfies the requirement mixtend (from versions: none)
ERROR: No matching distribution found for mixtend
You should consider upgrading via the 'c:\programdata\anaconda3\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [4]:
# 리스트의 리스트 형태로 트랜잭션 구성
dataset = [
    ['빵', '우유', '씨리얼'],
    ['빵', '기저귀', '맥주', '계란'],
    ['우유', '기저귀', '맥주', '콜라'],
    ['빵', '우유', '기저귀', '맥주'],
    ['빵', '우유', '기저귀', '콜라', '삼겹살'],
    ['우유', '계란', '씨리얼'],
    ['우유', '기저귀', '콜라', '맥주'],
    ['삼겹살', '콜라', '맥주'],
    ['빵', '우유', '기저귀', '맥주', '계란'],
    ['우유', '빵'],
    ['우유', '씨리얼']
]

In [6]:
# TransactionEncoder는 리스트의 리스트로 구성된 트랜잭션 데이터를 numpy 배열로 인코딩
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
te_ary

array([[False, False, False,  True, False,  True,  True, False],
       [ True,  True,  True,  True, False, False, False, False],
       [False,  True,  True, False, False, False,  True,  True],
       [False,  True,  True,  True, False, False,  True, False],
       [False,  True, False,  True,  True, False,  True,  True],
       [ True, False, False, False, False,  True,  True, False],
       [False,  True,  True, False, False, False,  True,  True],
       [False, False,  True, False,  True, False, False,  True],
       [ True,  True,  True,  True, False, False,  True, False],
       [False, False, False,  True, False, False,  True, False],
       [False, False, False, False, False,  True,  True, False]])

- 원핫인코딩과 유사한 형태로 변환됨

In [9]:
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,계란,기저귀,맥주,빵,삼겹살,씨리얼,우유,콜라
0,False,False,False,True,False,True,True,False
1,True,True,True,True,False,False,False,False
2,False,True,True,False,False,False,True,True
3,False,True,True,True,False,False,True,False
4,False,True,False,True,True,False,True,True
5,True,False,False,False,False,True,True,False
6,False,True,True,False,False,False,True,True
7,False,False,True,False,True,False,False,True
8,True,True,True,True,False,False,True,False
9,False,False,False,True,False,False,True,False


In [10]:
# 지지도확인
# 디폴트는 support가 0.5이상인 데이터만 보여줌

frequent_itemsets = apriori(df, use_colnames = True)
frequent_itemsets

,support,itemsets
0,0.545455,(기저귀)
1,0.545455,(맥주)
2,0.545455,(빵)
3,0.818182,(우유)


In [11]:
# support 0.3 이상

frequent_itemsets = apriori(df, min_support=0.3, use_colnames = True)
frequent_itemsets

,support,itemsets
0,0.545455,(기저귀)
1,0.545455,(맥주)
2,0.545455,(빵)
3,0.818182,(우유)
4,0.363636,(콜라)
5,0.454545,"(맥주, 기저귀)"
6,0.363636,"(빵, 기저귀)"
7,0.454545,"(우유, 기저귀)"
8,0.363636,"(맥주, 우유)"
9,0.454545,"(빵, 우유)"


In [14]:
# support 내림차순

frequent_itemsets.sort_values('support', ascending=False)

,support,itemsets
3,0.818182,(우유)
0,0.545455,(기저귀)
1,0.545455,(맥주)
2,0.545455,(빵)
5,0.454545,"(맥주, 기저귀)"
7,0.454545,"(우유, 기저귀)"
9,0.454545,"(빵, 우유)"
4,0.363636,(콜라)
6,0.363636,"(빵, 기저귀)"
8,0.363636,"(맥주, 우유)"


In [15]:
# 신뢰도 확인 (디폴트 0.8 이상)
# metric='confidence'

association_rules(frequent_itemsets, metric='confidence')

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(맥주),(기저귀),0.545455,0.545455,0.454545,0.833333,1.527778,0.157025,2.727273
1,(기저귀),(맥주),0.545455,0.545455,0.454545,0.833333,1.527778,0.157025,2.727273
2,(기저귀),(우유),0.545455,0.818182,0.454545,0.833333,1.018519,0.008264,1.090909
3,(빵),(우유),0.545455,0.818182,0.454545,0.833333,1.018519,0.008264,1.090909
4,"(맥주, 우유)",(기저귀),0.363636,0.545455,0.363636,1.000000,1.833333,0.165289,inf
5,"(기저귀, 우유)",(맥주),0.454545,0.545455,0.363636,0.800000,1.466667,0.115702,2.272727
6,"(맥주, 기저귀)",(우유),0.454545,0.818182,0.363636,0.800000,0.977778,-0.008264,0.909091


In [16]:
# 신뢰도 확인 (디폴트 0.8 이상)
# metric='confidence'
# min_threshold 지정 신뢰도 필터링

association_rules(frequent_itemsets, metric='confidence', min_threshold=0.3)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(맥주),(기저귀),0.545455,0.545455,0.454545,0.833333,1.527778,0.157025,2.727273
1,(기저귀),(맥주),0.545455,0.545455,0.454545,0.833333,1.527778,0.157025,2.727273
2,(빵),(기저귀),0.545455,0.545455,0.363636,0.666667,1.222222,0.066116,1.363636
3,(기저귀),(빵),0.545455,0.545455,0.363636,0.666667,1.222222,0.066116,1.363636
4,(우유),(기저귀),0.818182,0.545455,0.454545,0.555556,1.018519,0.008264,1.022727
5,(기저귀),(우유),0.545455,0.818182,0.454545,0.833333,1.018519,0.008264,1.090909
6,(맥주),(우유),0.545455,0.818182,0.363636,0.666667,0.814815,-0.082645,0.545455
7,(우유),(맥주),0.818182,0.545455,0.363636,0.444444,0.814815,-0.082645,0.818182
8,(빵),(우유),0.545455,0.818182,0.454545,0.833333,1.018519,0.008264,1.090909
9,(우유),(빵),0.818182,0.545455,0.454545,0.555556,1.018519,0.008264,1.022727


복수개의 품목에 대한 조합도 식별 가능

In [18]:
# 향상도 확인
# 1이 넘어야 의미가 있음

association_rules(frequent_itemsets, metric='lift', min_threshold=1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(맥주),(기저귀),0.545455,0.545455,0.454545,0.833333,1.527778,0.157025,2.727273
1,(기저귀),(맥주),0.545455,0.545455,0.454545,0.833333,1.527778,0.157025,2.727273
2,(빵),(기저귀),0.545455,0.545455,0.363636,0.666667,1.222222,0.066116,1.363636
3,(기저귀),(빵),0.545455,0.545455,0.363636,0.666667,1.222222,0.066116,1.363636
4,(우유),(기저귀),0.818182,0.545455,0.454545,0.555556,1.018519,0.008264,1.022727
5,(기저귀),(우유),0.545455,0.818182,0.454545,0.833333,1.018519,0.008264,1.090909
6,(빵),(우유),0.545455,0.818182,0.454545,0.833333,1.018519,0.008264,1.090909
7,(우유),(빵),0.818182,0.545455,0.454545,0.555556,1.018519,0.008264,1.022727
8,"(맥주, 우유)",(기저귀),0.363636,0.545455,0.363636,1.000000,1.833333,0.165289,inf
9,"(기저귀, 우유)",(맥주),0.454545,0.545455,0.363636,0.800000,1.466667,0.115702,2.272727


In [19]:
df_assoc = association_rules(frequent_itemsets, metric='lift', min_threshold=1.2)
df_assoc.sort_values('lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,"(맥주, 우유)",(기저귀),0.363636,0.545455,0.363636,1.000000,1.833333,0.165289,inf
7,(기저귀),"(맥주, 우유)",0.545455,0.363636,0.363636,0.666667,1.833333,0.165289,1.909091
0,(맥주),(기저귀),0.545455,0.545455,0.454545,0.833333,1.527778,0.157025,2.727273
1,(기저귀),(맥주),0.545455,0.545455,0.454545,0.833333,1.527778,0.157025,2.727273
5,"(기저귀, 우유)",(맥주),0.454545,0.545455,0.363636,0.800000,1.466667,0.115702,2.272727
6,(맥주),"(기저귀, 우유)",0.545455,0.454545,0.363636,0.666667,1.466667,0.115702,1.636364
2,(빵),(기저귀),0.545455,0.545455,0.363636,0.666667,1.222222,0.066116,1.363636
3,(기저귀),(빵),0.545455,0.545455,0.363636,0.666667,1.222222,0.066116,1.363636


- (우유, 맥주) 를 구매했을 때 (기저귀) 추천
- (기저귀) 를 구매했을 때 (맥주, 우유) 추천
등등 유의미한 결과를 확인